In [49]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

In [50]:
today = date.today()
d1 = today.strftime("%Y-%m-%d")

In [51]:
invoice_df=pd.read_csv("invoice_daily_actuals.csv")
phone_df=pd.read_csv("phone_daily_actuals.csv")
invoice_df['date']=pd.to_datetime(invoice_df['effective_date'] , format='%Y%m%d')
phone_df['date']=pd.to_datetime(phone_df['effective_date'], format='%Y-%m-%d')

invoice_df=invoice_df.rename(columns={'actual': 'invoice_actual'})
phone_df=phone_df.rename(columns={'actual': 'phone_actual'})

In [52]:
combine_list=np.concatenate((invoice_df.store_code.unique(), phone_df.store_code.unique()))
combine_list = list(filter(None, combine_list))
combine_list=np.unique(combine_list, axis=0)

invoice_max_date=max(invoice_df['date'])
phone_max_date=max(phone_df['date'])
max_date = invoice_max_date if invoice_max_date >= phone_max_date else phone_max_date

df1 = pd.merge(
    pd.DataFrame(combine_list,columns=["store_code"]).assign(key=1),
    pd.DataFrame(pd.date_range(start="2017-01-01", end=d1), columns=["date"]).assign(key=1),
    how='outer').drop('key',1)

df1=df1[df1['date']<=max_date]
df1=pd.merge(df1, invoice_df[['store_code','date','invoice_actual']], on=['store_code','date'], how='left')
df1=pd.merge(df1, phone_df[['store_code','date','phone_actual']], on=['store_code','date'], how='left')
df1['year']=pd.to_datetime(df1['date']).dt.year
df1['month']=pd.to_datetime(df1['date']).dt.month
df1['month']=df1.month.map("{:02}".format)
df1['key']=df1['year'].astype(str) + df1['month'].astype(str)
df1['key']=df1['key'].astype(int)

In [53]:
##for invoice
df2 = df1.groupby(['store_code','key'])['invoice_actual'].sum()
df2 = df2.reset_index()
df2.sort_values(by=['store_code','key'], inplace=True)
df2['p_m'] = df2.groupby(['store_code'])['invoice_actual'].shift(1)
df2['p_y_m'] = df2.groupby(['store_code'])['invoice_actual'].shift(13)
df2['invoice_prev_month_yoy']=(df2['p_m']-df2['p_y_m'])/df2['p_y_m']
df2['missing_invoice_prev_month_yoy'] = np.where(df2.invoice_prev_month_yoy.isnull(),1, 0)

In [54]:
##for phone
df3 = df1.groupby(['store_code','key'])['phone_actual'].sum()
df3 = df3.reset_index()
df3.sort_values(by=['store_code','key'], inplace=True)
df3['p_m'] = df3.groupby(['store_code'])['phone_actual'].shift(1)
df3['p_y_m'] = df3.groupby(['store_code'])['phone_actual'].shift(13)
df3['phone_prev_month_yoy']=(df3['p_m']-df3['p_y_m'])/df3['p_y_m']
df3['missing_phone_prev_month_yoy'] = np.where(df3.phone_prev_month_yoy.isnull(),1, 0)
#df3

In [55]:
df1=pd.merge(df1, df2[['store_code','key','invoice_prev_month_yoy','missing_invoice_prev_month_yoy']], on=['store_code','key'], how='left')
df1=pd.merge(df1, df3[['store_code','key','phone_prev_month_yoy','missing_phone_prev_month_yoy']], on=['store_code','key'], how='left')
df1['invoice_prev_month_yoy']=df1['invoice_prev_month_yoy'].fillna(0)
df1['phone_prev_month_yoy']=df1['phone_prev_month_yoy'].fillna(0)
df1=df1[['store_code','date','invoice_prev_month_yoy','phone_prev_month_yoy','missing_invoice_prev_month_yoy','missing_phone_prev_month_yoy']]

In [57]:
df1.to_csv('yoy.csv')